In [ ]:
# Download training data
# !curl -O http://static.decontextualize.com/gutenberg-poetry-v001.ndjson.gz

In [ ]:
import tensorflow as tf
import numpy as np 

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from livelossplot.tf_keras import PlotLossesCallback

In [ ]:
# Unzip and read data
import gzip, json
all_lines = []
for line in gzip.open("./gutenberg-poetry-v001.ndjson.gz"):
    all_lines.append(json.loads(line.strip()))

In [ ]:
#prepare data
poem_lines = [line['s'] for line in all_lines[23:20000]] # Reading only till 200000 lines to save memory

In [ ]:
# Tokenizer
tokenizer = Tokenizer()
# fit tokenizer on corpus
tokenizer.fit_on_texts(poem_lines)

In [ ]:
total_words = len(tokenizer.word_index) + 1 
print(total_words)

In [ ]:
input_sequences = []
for line in poem_lines:
    token_list = tokenizer.texts_to_sequences([line])[0]
    # print(f'{token_list} Token list')
    for i in range(1, len(token_list)):
        ngrams = token_list[:i+1]
        input_sequences.append(ngrams)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
# Adjusting input_sequence size to save memory
# input_sequences = input_sequences[:10000] 

In [ ]:
# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)
# ys = labels

In [ ]:
print(xs[5])
print(ys[5])

In [ ]:
# tokenizer.word_index

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 1000, input_length=max_sequence_len-1))

model.add(Bidirectional(LSTM(256,return_sequences=True )))
model.add(Bidirectional(LSTM(128,return_sequences=True )))
model.add(Bidirectional(LSTM(64)))

model.add(Dense(total_words//3, activation='relu'))
# model.add(Dense(total_words//2, activation='relu'))
model.add(Dense(total_words, activation='softmax'))

adam = Adam(lr=0.03)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])


print (model.summary())

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.3,
                              patience=5, min_lr=0.00001, mode='auto', verbose=1)

checkpoint = ModelCheckpoint("saved_weights.h5", monitor='accuracy',
                             save_weights_only=True, mode='max', verbose=1)



earlystop = EarlyStopping(monitor='accuracy', min_delta=0, patience=5, verbose=1, mode='max')

callbacks = [PlotLossesCallback(), checkpoint, reduce_lr, earlystop]

history = model.fit(xs, ys, epochs=100, verbose=1, callbacks=callbacks)